### utilities

In [ ]:
import random
import math
import csv
import random
import numpy as np
import sys
import pandas as pd
import torch
from itertools import combinations

import pickle

from collections import defaultdict, Counter
from sklearn.cluster import OPTICS, cluster_optics_dbscan

import torch.nn as nn
import torch.nn.functional as F
import copy

def accuracy(workerlist,threshold,t2truth):
  
  tasklist = []
  worker_acc_dict = {}
  for w in workerlist:
    
    total_tuple_list = w2tl[w]
    

    if len(total_tuple_list) > threshold:
      short_task_list = random.sample(total_tuple_list, 5)
    
    acc =0
    for t_id,w_answer in total_tuple_list:
      
      truth = t2truth[t_id]
      if w_answer == truth:
        acc = acc+1
      
    acc = acc/len(total_tuple_list)
    worker_acc_dict[w] = acc
  
  return worker_acc_dict

def accuracy_all(workerlist,t2truth):
  
  worker_acc_dict = {}
  for w in workerlist:
    
    total_tuple_list = w2tl[w]
    
    acc =0
    for t_id,w_answer in total_tuple_list:
      
      truth = t2truth[t_id]
      if w_answer == truth:
        acc = acc+1
      
    acc = acc/len(total_tuple_list)
    worker_acc_dict[w] = acc
  
  return worker_acc_dict
def worker_cluster_acc(ep_label_woker_dict,worker_acc_dict):
  cluster_acc_dict={}
  for ep in ep_label_woker_dict:
    label_wid_dict = ep_label_woker_dict[ep]
  
  for label in label_list:
    worker_list = label_wid_dict[label]
    
    acc = 0
    for wid in worker_list:
      
      acc_w = worker_acc_dict[wid]
      acc = acc + acc_w
    acc = acc/len(worker_list)
    cluster_acc_dict[label] = acc
  
  return cluster_acc_dict


def input2numpy(input_dict,wstrid2wnumid,wnumid2wstrid,feat_size,path,name):
  x_train = np.zeros((feat_size,),dtype = np.float)
  for i in range(len(input_dict)):
    
    str_id  = wnumid2wstrid[i]
    
    worker_input_array = np.asarray(input_dict[str_id])
    x_train = np.vstack((x_train,worker_input_array))

  x_train_all = np.delete(x_train,0,axis=0)
  np.save(path+name,x_train_all)
  return x_train_all

def extend_nodes(nodes_batch, training_cps, wstrid2wnumid):
  
  original_nodes_batch = copy.deepcopy(nodes_batch)
  node_pairs = {}
  extended_nodes_batch = set(nodes_batch)
  
  for node in nodes_batch:
      strid = wnumid2wstrid[node]
      cps = training_cps[strid]
      node_pairs[node] = cps
      for cp in cps:
        extended_nodes_batch.add(cp[1])
  extended_nodes_batch = list(extended_nodes_batch)
  return extended_nodes_batch

def w2wsim2numpy(input_dict,wstrid2wnumid,wnumid2wstrid,feat_size,path,name):
  x_train = np.zeros((feat_size,),dtype = np.float)
  for i in range(len(input_dict)):
    
    str_id  = wnumid2wstrid[i]
    
    worker_input_array = np.asarray(input_dict[str_id])

    
    worker_input_array[worker_input_array<0.3] = 0
    worker_input_array[worker_input_array>=0.3] = 1

    x_train = np.vstack((x_train,worker_input_array))

  x_train_all = np.delete(x_train,0,axis=0)
  np.save(path+name,x_train_all)
  return x_train_all

### TWLR

In [ ]:
class TWLR(nn.Module):
    def __init__(self, w_pb_input, feat_size, emb_size):
   
        super(TWLR, self).__init__()
        self.w_pb_input = w_pb_input
        
        self.fc1 = nn.Linear(feat_size, 64)
        self.fc2 = nn.Linear(64, emb_size)
        self.fc3 = nn.Linear(emb_size, 64)
        self.fc4 = nn.Linear(64, feat_size)
        
    def init_params(self):
        for param in self.parameters():
            if len(param.size()) == 2:
                nn.init.xavier_uniform_(param)
            else:
                # initialize all bias as zeros
                nn.init.constant_(param, 0.0)
    
    def forward(self, nodes_batch):
       
        feats = self.w_pb_input[nodes_batch]

        feats = feats + torch.rand(feats.shape)
        x_en = F.relu_(self.fc1(feats))
        embs = F.relu_(self.fc2(x_en))
        x_de = F.relu_(self.fc3(embs))
        recon = F.relu_(self.fc4(x_de))

        return embs, recon

In [ ]:
# 20200824 加了一个layersize
class TWLR(nn.Module):
    def __init__(self, w_pb_input, feat_size, layersize, emb_size):
    #def __init__(self, features, feat_size, hidden_size, emb_size):
        super(TWLR, self).__init__()
        self.w_pb_input = w_pb_input
        # feat_size 就是输入的size, hidden_size就是第一次变换的size,emb_size就是encoder输出的size;
        self.fc1 = nn.Linear(feat_size, layersize)
        self.fc2 = nn.Linear(layersize, emb_size)
        self.fc3 = nn.Linear(emb_size, layersize)
        self.fc4 = nn.Linear(layersize, feat_size)
        
    def init_params(self):
        for param in self.parameters():
            if len(param.size()) == 2:
                nn.init.xavier_uniform_(param)
            else:
                # initialize all bias as zeros
                nn.init.constant_(param, 0.0)
    # forward 的参数, nodes_batch 就是 forward 函数的 input; 
    # nodes_batch 就是一组数据,nodes_batch 是编号, features.nodes_batch 才是数据
    def forward(self, nodes_batch):
        feats = self.w_pb_input[nodes_batch]
        x_en = F.relu_(self.fc1(feats))
        embs = F.relu_(self.fc2(x_en))
        x_de = F.relu_(self.fc3(embs))
        recon = F.relu_(self.fc4(x_de))
        #20200716 换成sigmoid 激活函数
        # recon = F.sigmoid(self.fc4(x_de))
        return embs, recon

###TWLR loss

In [ ]:

class Loss_TWLR():
    def __init__(self, features, worker_similarity_df, wnumid2wstrid, training_cps, device, alpha, beta, gamma):
        self.worker_similarity_df = worker_similarity_df
        self.wnumid2wstrid = wnumid2wstrid
        

        self.training_cps = training_cps

        self.features = features
        
        self.device = device

        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma

        
        
        self.node_pairs = {}
        self.original_nodes_batch = None
        self.extended_nodes_batch = None


    def extend_nodes(self, nodes_batch):
        
        self.original_nodes_batch = copy.deepcopy(nodes_batch)
        self.node_pairs = {}
        self.extended_nodes_batch = set(nodes_batch)
        
        for node in nodes_batch:
            strid = self.wnumid2wstrid[node]
            cps = self.training_cps[strid]
            self.node_pairs[node] = cps
            for cp in cps:
              self.extended_nodes_batch.add(cp[1])
        self.extended_nodes_batch = list(self.extended_nodes_batch)
        return self.extended_nodes_batch
    
    def get_loss(self, nodes_batch, embs_batch, recon_batch):
        
        loss_recon = self.get_loss_recon(nodes_batch, recon_batch)
        loss_simi = self.get_loss_simi(embs_batch)
        loss = loss_recon + self.alpha * loss_simi
        return loss
    
    
    def get_loss_simi(self, embs_batch):
        
        node2index = {n:i for i,n in enumerate(self.extended_nodes_batch)}
        
        simi_feat = []
        simi_embs = []
        
        
        for node, cps in self.node_pairs.items():
            for i, j in cps:
              
              wi=wnumid2wstrid[i]
              wj=wnumid2wstrid[j]
              
              wistrid = 'w' + str(wi)
              wjstrid = 'w' + str(wj)
              simi = self.worker_similarity_df.loc[wistrid,wjstrid]

              simi_feat.append(torch.FloatTensor([simi]))
              dis_ij = (embs_batch[node2index[i]] - embs_batch[node2index[j]]) ** 2
              dis_ij = torch.exp(-dis_ij.sum())
              
              simi_embs.append(dis_ij.view(1))
        simi_feat = torch.cat(simi_feat, 0).to(self.device)
        simi_embs = torch.cat(simi_embs, 0)
        L = simi_feat * ((simi_embs - simi_feat) ** 2)
        return L.mean()

    def get_loss_recon(self, nodes_batch, recon_batch):
        feats_batch = self.features[nodes_batch]
        
        H_batch = (feats_batch * (self.beta - 1)) + 1
        assert feats_batch.size() == recon_batch.size() == H_batch.size()
        
        L = ((recon_batch - feats_batch) * H_batch) ** 2
        return L.mean()

### train 



In [ ]:

def train_model(wnumid2wstrid,training_cps, TWLR, model_loss, device, epoch):
 
    train_nodes = np.arange(len(wnumid2wstrid))

    np.random.shuffle(train_nodes)

    
    params = []
    
    learning_rate = 0.025
    
    gama = 0.001
    
    for param in TWLR.parameters():
        if param.requires_grad:
            params.append(param)
    
    optimizer = torch.optim.SGD(params, lr=lr, weight_decay=gamma)
    
    optimizer.zero_grad()
    TWLR.zero_grad()
  
    b_sz = 22
    batches = math.ceil(len(train_nodes) / b_sz)

    visited_nodes = set()
    
    training_cps = training_cps
    
    #logger.info('sampled pos and neg nodes for each node in this epoch.')
    for index in range(batches):
        
        nodes_batch = train_nodes[index*b_sz:(index+1)*b_sz]
        
        nodes_batch = np.asarray(model_loss.extend_nodes(nodes_batch))
        
        visited_nodes |= set(nodes_batch)

        embs_batch, recon_batch = TWLR(nodes_batch)
        loss = model_loss.get_loss(nodes_batch, embs_batch, recon_batch)

        loss.backward()

        nn.utils.clip_grad_norm_(TWLR.parameters(), 5)
        optimizer.step()

        optimizer.zero_grad()
        TWLR.zero_grad()

        #输入一下loss
        print('epoch', epoch,'batch',index, 'loss =',loss.data.numpy())
        # stop when all nodes are trained
        if len(visited_nodes) == len(train_nodes):
            break

    return TWLR 

## get_embeddings

In [ ]:
def get_embeddings(TWLR, wnumid2wstrid, path):
  train_nodes = np.arange(len(wnumid2wstrid))
  with torch.no_grad():
    embs_batch, _ = TWLR(train_nodes)
  assert len(embs_batch) == len(train_nodes)
  embs_temp = embs_batch.detach()
  worker_emb_numpy = embs_temp.numpy()
  np.save(path+'worker_emb_numpy.npy',worker_emb_numpy)
  
  return worker_emb_numpy
    
    



In [ ]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from itertools import cycle
from sklearn.cluster import OPTICS, cluster_optics_dbscan
def distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, deepSW):
  
  
  worker_emb_numpy = get_embeddings(deepSW, wnumid2wstrid, path)
  w2sumdistance_dict = {}
 
  for i in range(worker_emb_numpy.shape[0]):
    temp = 0
    for j in range(worker_emb_numpy.shape[0]):
     
      dists = 1 - cosine_similarity(worker_emb_numpy[i].reshape(1,-1), worker_emb_numpy[j].reshape(1,-1))
      temp = temp + dists
   
    w2sumdistance_dict[i] = temp[0][0]
  
  return w2sumdistance_dict

  
  

## Top1_experts


In [ ]:
def take3rd(elem):
    return elem[2]
def Top1_experts(w2sumdistance_dict,wstrid2wnumid, wnumid2wstrid, t2wl):

  t2pwld = {}
  for t in t2wl:
    wl_list = t2wl[t]
    w1d_list = []
    predict = 0
    
    for wl_tuple in wl_list:
      worker = wl_tuple[0]
      label = wl_tuple[1]
      numid = wstrid2wnumid[worker]
      w_dis = w2sumdistance_dict[numid]
      w1d_list.append([worker,label,w_dis])

    w1d_list.sort(key=take3rd,reverse=False)
   
    t2pwld[t] = w1d_list[0:1]

  return t2pwld
  

In [ ]:
def wstr_2_distance(w2sumdistance_dict,wstrid2wnumid):


  wstr2distance = {}
  for wnumid in w2sumdistance_dict:
    wstrid = wnumid2wstrid[wnumid]
    wstr2distance[wstrid] = w2sumdistance_dict[wnumid]

  return wstr2distance

In [ ]:
def getaccuracy_Top1(e2truth, t2pwld, t2w):
  
  count = 0
  right_taskid_list = []
  wrong_taskid_list = []
 
  for e in t2w:

    if e not in e2truth:
      continue

    temp = t2pwld[e]
    
    predict_answer = temp[0][1]
  
    count += 1 
    
    if predict_answer == e2truth[e]:
      
      right_taskid_list.append(e)
    else:
      wrong_taskid_list.append(e)  

  
  return len(right_taskid_list)/count, right_taskid_list, wrong_taskid_list

## main 


In [ ]:
import random
import math
import csv
import random
import numpy as np
import sys
import pandas as pd
import torch
from itertools import combinations

import pickle

from collections import defaultdict, Counter
from sklearn.cluster import OPTICS, cluster_optics_dbscan

import torch.nn as nn
import torch.nn.functional as F
import copy


In [ ]:

save_path = '/content/drive/MyDrive/Colab Notebooks/Deep worker cluster/files_for_IJCAI2021/data/'
pathlist = [
        save_path +'active-crowd-toolkit/MS/',
         save_path +'active-crowd-toolkit/ZenCrowd_us/',
         save_path +'SpectralMethodsMeetEM/bluebird/',
         save_path +'SpectralMethodsMeetEM/trec/',
         save_path +'crowdscale2013/fact_eval/',
         save_path +'crowd_truth_inference/d_Duck Identification/',
         save_path +'crowd_truth_inference/s4_Face Sentiment Identification/',
        
          ]

In [ ]:

alpha_list = [0,10,20,30,40,50]
emb_size_list= [2,3,4,5,6,7,8,9,10]
layer_size_list = [16,32,64]
output_path = '/content/drive/MyDrive/Colab Notebooks/Deep worker cluster/files_for_IJCAI2021/'

for path in pathlist:
  seed = 1234
  device = torch.device('cpu')
  
  hidden_size = 128
  emb_size = 4
  epochs = 50
  
  beta = 20
  gamma = 0.01
  lr = 0.025

  with open(path+'w2task_input.pickle','rb') as file:
    w2task_input = pickle.load(file)

  with open(path+'wnumid2wstrid.pickle','rb') as file:
    wnumid2wstrid = pickle.load(file)

  with open(path+'wstrid2wnumid.pickle','rb') as file:
    wstrid2wnumid = pickle.load(file)

  with open(path+'t2w.pickle','rb') as file:
    t2w = pickle.load(file)
  with open(path+'t2wl.pickle','rb') as file:
    t2wl = pickle.load(file)


  with open(path+'t2truth.pickle','rb') as file:
    t2truth = pickle.load(file)


  with open(path+'w2t.pickle','rb') as file:
    w2t = pickle.load(file)

  with open(path+'w2tl.pickle','rb') as file:
    w2tl = pickle.load(file)

  with open(path+'worker_acc_dict.pickle','rb') as file:
    worker_acc_dict = pickle.load(file)



  with open(path+'w2w.pickle','rb') as file:
    w2w = pickle.load(file) 

  with open(path+'training_cps.pickle','rb') as file:
    training_cps = pickle.load(file) 


  with open(path+'label_set.pickle','rb') as file:
    label_set = pickle.load(file)

 
  csv_data = pd.read_csv(path+'worker_simlarity.csv',low_memory = False,index_col=0)
  worker_similarity_df = pd.DataFrame(csv_data)

  datasetname = path.split('/')[-2]
  output_file = output_path + datasetname+ '_noise_WORT_TOP1_MV_answer_3.csv'
  

  for apa in alpha_list:
    
    for layer in layer_size_list:
      for emb_size in emb_size_list:
    
        
        
        output_list =[]
        output_list.append(apa)
        output_list.append(layer)
        output_list.append(emb_size)

        alpha = apa  

        pb_feat_size = len(t2w)
        w_pb_input = input2numpy(w2task_input,wstrid2wnumid,wnumid2wstrid,pb_feat_size,path,'w_pb_input.npy')
        w_pb_input_tensor = torch.FloatTensor(w_pb_input).to(device)
 
        TWLR = TWLR(w_pb_input_tensor, w_pb_input_tensor.size(1),layer, emb_size)
        TWLR.to(device)
        
        model_loss = loss_TWLR(w_pb_input_tensor, worker_similarity_df, wnumid2wstrid, training_cps, device, alpha, beta, gamma)
        max_temp_Top1 = 0
        acc_Top1_all = 0 

        for epoch in range(epochs):
          #logger.info(f'----------------------EPOCH {epoch}-----------------------')
          
          TWLR = train_model(wnumid2wstrid, training_cps, TWLR, model_loss, device, epoch)
        
          if (epoch+1) % 10 == 0:    
            w2sumdistance_dict = distance_experts(wstrid2wnumid, wnumid2wstrid, worker_acc_dict, t2w, t2wl, w2t, path, TWLR)
            
            t2pwld  = Top1_experts(w2sumdistance_dict, wstrid2wnumid, wnumid2wstrid, t2wl)
            acc_Top1, rightlist_Top1, wronglist_Top1  = getaccuracy_Top1(t2truth, t2pwld, t2w)

            output_list.append(acc_Top1)
            output_list.append(len(rightlist_Top1))
            output_list.append(len(wronglist_Top1))

            acc_Top1_all = acc_Top1_all + acc_Top1
            
            if acc_Top1 > max_temp_Top1:
              max_temp_Top1 = acc_Top1

            if (epoch+1) % 50 == 0:
              acc_Top1_all = acc_Top1_all / 5
              output_list.append(acc_Top1_all)
              output_list.append(max_temp_Top1)

              with open(output_file,'a+')as f:
                f_csv = csv.writer(f)
                f_csv.writerow(output_list)
              

流式输出内容被截断，只能显示最后 5000 行内容。
epoch 4 batch 14 loss = 2.4539053
epoch 4 batch 15 loss = 2.3867378
epoch 4 batch 16 loss = 2.4544718
epoch 4 batch 17 loss = 2.487422
epoch 4 batch 18 loss = 2.3173964
epoch 5 batch 0 loss = 2.3350198
epoch 5 batch 1 loss = 2.5833275
epoch 5 batch 2 loss = 2.3196468
epoch 5 batch 3 loss = 2.458021
epoch 5 batch 4 loss = 2.2637415
epoch 5 batch 5 loss = 2.5290709
epoch 5 batch 6 loss = 2.342804
epoch 5 batch 7 loss = 2.4007227
epoch 5 batch 8 loss = 2.1097195
epoch 5 batch 9 loss = 2.524586
epoch 5 batch 10 loss = 2.2316613
epoch 5 batch 11 loss = 2.575252
epoch 5 batch 12 loss = 2.4171696
epoch 5 batch 13 loss = 2.3164313
epoch 5 batch 14 loss = 2.2050302
epoch 5 batch 15 loss = 2.6282492
epoch 6 batch 0 loss = 2.374275
epoch 6 batch 1 loss = 2.4312558
epoch 6 batch 2 loss = 2.5043182
epoch 6 batch 3 loss = 2.6227229
epoch 6 batch 4 loss = 2.3664658
epoch 6 batch 5 loss = 2.4877694
epoch 6 batch 6 loss = 2.371396
epoch 6 batch 7 loss = 2.6546268
epoch 6 batc